In [2]:
import pandas as pd

Ethiopia = pd.read_csv("ethiopia_df.csv")
Malawi = pd.read_csv("malawi_df.csv")
Mali = pd.read_csv("mali_df.csv")
Nigeria = pd.read_csv("nigeria_df.csv")

Ethiopia = Ethiopia.drop('Unnamed: 0', axis=1)
Malawi = Malawi.drop('Unnamed: 0', axis=1)
Mali = Mali.drop('Unnamed: 0', axis=1)
Nigeria = Nigeria.drop('Unnamed: 0', axis=1)

imageVals = list(Ethiopia['Image'].values) + list(Malawi['Image'].values) + list(Mali['Image'].values) + list(Nigeria['Image'].values)
PPPvals = list(Ethiopia['PPP'].values) + list(Malawi['PPP'].values) + list(Mali['PPP'].values) + list(Nigeria['PPP'].values)
PPPvals = [int(i*1000+.5) for i in PPPvals]

data = []
for i in range(len(PPPvals)):
    data.append([imageVals[i],PPPvals[i]])

data = pd.DataFrame(data, columns=['Image', 'PPP'])

In [3]:
print(sum(val < 1500 for val in PPPvals))
print(sum(1500 <= val < 2000 for val in PPPvals))
print(sum(2000 <= val < 3000 for val in PPPvals))
print(sum(3000 <= val for val in PPPvals)) #70-30 and 75-25 train-test split

11810
10363
12908
12340
47421


In [13]:
import numpy as np

PPPclassification = []
for val in PPPvals:
    if val<1500:
        PPPclassification.append(0) 
    elif 1500<val<2000:
        PPPclassification.append(1) 
    elif 2000<val<3000:
        PPPclassification.append(2) 
    else:
        PPPclassification.append(3) 

data['Class'] = PPPclassification

In [14]:
import matplotlib.pyplot as plt
import numpy as np

imgArrays = []

#count=0

for img in imageVals:
    
    array = plt.imread(img, format=None)

    imgList = []
    for i in array:
        list1 = []
        for j in i:
            list2 = [j[0],j[1],j[2]]
            list1.append(list2)
        imgList.append(list1)
    array = imgList
    imgArrays.append(array)
'''
    count+=1
    if count == 100:
        break
'''
imgArrays = np.array(imgArrays)
classes = np.array(data['Class'].values)

%store imgArrays

In [1]:
from sklearn.model_selection import train_test_split
%store -r imgArrays
classes = np.array(data['Class'].values)


X = imgArrays
y = classes #y = classes[0:100]

print(len(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=2)
'''
y_train_encoded = []
for i in y_train:
    temp = [0,0,0,0]
    temp[i]=1
    y_train_encoded.append(temp)


y_test_encoded = []
for i in y_test:
    temp = [0,0,0,0]
    temp[i]=1
    y_test_encoded.append(temp)
'''

NameError: name 'np' is not defined

In [ ]:
import tensorflow as tf
from keras import layers
from keras import models

/Users/dalar25/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/dalar25/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/dalar25/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/dalar25/opt/anaconda3/envs/tf/lib

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) #activation function relu takes the sum of all inputs, but if it is negative, the output is 0
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu')) #Dense: all previous nodes connected to new nodes
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(16, activation='relu')) 
model.add(layers.Dense(1))
#model.add(layers.Activation('softmax')) #softmax used with several classes and it gives probabilities for each class

#add dropout which makes sure some of the lines are not activated

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)               

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error')
#model.compile(optimizer='adam',
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #metrics=['accuracy'])

#y_test_encoded = np.array(y_test_encoded)
#y_train_encoded = np.array(y_train_encoded)
#X_train = np.array(X_train)
#X_test = np.array(X_test)

history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_test, y_test))

#history = model.fit(X_train, y_train_encoded, epochs=10, 
                    #validation_data=(X_test, y_test_encoded))


#test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
listY_test = [int(i) for i in y_test]
preds = [round(float(i)) for i in list(model.predict(X_test))]
numCorrect = 0
for i in range(len(listY_test)):
    if listY_test[i]==preds[i]:
        numCorrect+=1
print(numCorrect/len(preds))

2022-12-22 15:55:22.471741: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 15:55:22.472756: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.



Train on 75 samples, validate on 25 samples
Epoch 1/10
75/75 [==============================] - 14s 191ms/step - loss: 3.7496 - val_loss: 0.7902
Epoch 2/10
75/75 [==============================] - 12s 165ms/step - loss: 0.6104 - val_loss: 0.4793
Epoch 3/10
75/75 [==============================] - 13s 169ms/step - loss: 0.4477 - val_loss: 0.4904
Epoch 4/10
75/75 [==============================] - 12s 164ms/step - loss: 0.4879 - val_loss: 0.4363
Epoch 5/10
75/75 [==============================] - 13s 168ms/step - loss: 0.3925 - val_loss: 0.5019
Epoch 6/10
75/75 [==============================] - 12s 162ms/step - loss: 0.4272 - val_loss: 0.4962
Epoch 7/10
75/75 [==============================] - 13s 170ms/step - loss: 0.4623 - val_loss: 0.4029
Epoch 8/10
75/75 [==============================] - 12s 166ms/step - loss: 0.3792 - val_loss: 0.4196
Epoch 9/10
75/75 [==============================] - 13s 167ms/step - loss: 0.4321 - val_loss: 0.3966
Epoch 10/10
75/75 [===========================

In [ ]:
import shap
background = X_train[np.random.choice(X_train.shape[0], 10, replace=False)]
# DeepExplainer to explain predictions of the model
explainer = shap.DeepExplainer(model, background)
# compute shap values
shap_values = explainer.shap_values(X_test[0:10])

/Users/dalar25/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


keras is no longer supported, please use tf.keras instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
#print(
#list(model.predict(X_train[0:1])).index(max(list(model.predict(X_train[0:1])))))

print(model.predict(X_train[0:1]).tolist()[0].index(max(model.predict(X_train[0:1]).tolist()[0])))

explainer = shap.DeepExplainer(model, background)
shap_values = explainer.shap_values(X_test[0:11])

0
